# Heirarchical Model Predictive Control: HVAC Example

In [1]:
using JuMP
using Plasmo
using Ipopt
using Plots

In [2]:
# Get Neighboring Rooms
# Assume rooms are in a row

# returns a list of vectors where the index of the list is the room and the vectors are the neighboring rooms
function getNeighboringRooms(room, nrooms)
    if room == 1
        return [2,0]
    elseif room == nrooms
        return [nrooms - 1,0]
    else
        a = [room - 1,room + 1]
        return a
    end
end


getNeighboringRooms (generic function with 1 method)

In [3]:
# Thermal Dynamics into and between rooms
rho_air = 1.225 # kg/m^3
V_room = 50 # m^3

Ci = rho_air*V_room # mass of room i kg
cp = 1012 # heat capacity of air in room (J/kg*K)
Ts = 16+273 # temp of supply air (K)
Toa = 293 # temp of outside air (K)
Rij = .004 # thermal resistance between rooms i and j (K/kW)
Rioa = .0057 # thermal resistance between room and outside (K/kW)
Pid = 1# disturbance term from weather forecast

#Ci*(dTi)=dt*(ui*cp*(Ts−Ti)+∑((Tj−Ti)/Rij+(Toa−Ti)/Rioa)+Pid)

1

In [4]:
function getUnodeIndex(nU, nL, Lnode_Index)
    return ceil(Int, Lnode_Index/(nL/nU))
end

println(getUnodeIndex(2, 6, 1))

1


In [5]:
function getLnodePosition(nU, nL, Lnode_index)
    return Int(Lnode_index/(nL/nU))
end

println(getLnodePosition(3, 12, 4))

1


In [29]:
function HMPC_HVAC(n_u, n_l, dtu, dtl, num_rooms)
# find constraints, objective variables, setpoints,
    # Constraints:
    Ti = [293, 297] # K
    Td_array = zeros(num_rooms)
    Td = 296
    for i in 1:num_rooms
        Td_array[i] = Td
    end # desired temp of rooms
    ui = 1 # must be within [.005, 5] # kg/s

    # Tuning Parameters
    c1 = 1
    c2 = 2

    # Objective variables
    # Ti must be within 20 and 24 

    # function inputs
    n_rooms = num_rooms
    N_U = n_u # upper level nodes
    N_L = n_l # lower level nodes

    # Sampling periods (seconds)
    dt_U = dtu
    dt_L = dtl

    # Time Horizons
    horiz_U = 24*60*60 # seconds
    horiz_L = 2*60*60 # seconds

    HVAC = Model(Ipopt.Optimizer)
    graph = OptiGraph()

    @optinode(graph, Unodes[1:N_U]) # make N_U upper layer nodes
    @optinode(graph, Lnodes[1:N_L]) # make N_L lower layer nodes

    # Upper Level Initialization
    for (i, Unode) in enumerate(Unodes)

        # array belonging to upper layer containing temps of each room in the node
        @variable(Unode, T_hat_U_array[1:n_rooms], start = 300)
            # row index: room number
            # column index: time

        # array belonging to upper layer containing airflowrate into each room
        @variable(Unode, u_hat_U_array[1:n_rooms], start = 1)
            # row index: room number
            # column index: time
        
        #Establish Nonlin Dummy Var
        @variable(Unode, uT_U_array[1:n_rooms])
        
        for (j, uVal) in enumerate(Unode[:uT_U_array])
            @NLconstraint(Unode, uVal == u_hat_U_array[j]*cp*(Ts - T_hat_U_array[j]))
        end
        
        @objective(Unode, Min, sum(.01*(T_hat_U_array .- Td).^2 .+ 100*(u_hat_U_array).^2))
    end

    # Lower Level Initailization
    for (i, Lnode) in enumerate(Lnodes)
        @variable(Lnode, 293 <= T_hat_L_array[1:n_rooms] <= 297, start = 295) # temp of each room
        @variable(Lnode, .005 <= u_hat_L_array[1:n_rooms] <= 5, start = 1) # airflowrate of each room
        
        #Establish Nonlin Dummy Var
        @variable(Lnode, uT_L_array[1:n_rooms])
        
        for (j, lVal) in enumerate(Lnode[:uT_L_array])
            @NLconstraint(Lnode, lVal == u_hat_L_array[j]*cp*(Ts - T_hat_L_array[j]))
        end
        
        @variable(Lnode, T_0_LVal_array[1:n_rooms], start = 295) # T_0 vals belonging to Lnodes

    end
    

    # Link Constraints between Unodes
    for Unode_Index in 1:(length(Unodes))
        for (room_num, T_room_num) in enumerate(Unodes[Unode_Index][:T_hat_U_array][:,1]) # only use temps of rooms at earliest time
            if Unode_Index <= length(Unodes) - 1
                if room_num == 1 # if in the first room
                    Tj = Unodes[Unode_Index][:T_hat_U_array][room_num + 1, 1] # establish neighboring room
                    @linkconstraint(graph, Unodes[Unode_Index + 1][:T_hat_U_array][room_num] == 
                        dt_U/Ci*(Unodes[Unode_Index][:uT_U_array][room_num] + ((Tj − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                        (Unodes[Unode_Index][:T_hat_U_array][room_num])) # relationship between room temps

                elseif room_num == n_rooms # if in the last last room
                    Tj = Unodes[Unode_Index][:T_hat_U_array][room_num - 1, 1]
                    @linkconstraint(graph, (Unodes[Unode_Index + 1][:T_hat_U_array][room_num]) == 
                        dt_U/Ci*(Unodes[Unode_Index][:uT_U_array][room_num] + ((Tj − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                        (Unodes[Unode_Index][:T_hat_U_array][room_num])) # relationship between room temps

                else # if in a room surrounded by 2 rooms
                    Tj1 = Unodes[Unode_Index][:T_hat_U_array][room_num - 1, 1]
                    Tj2 = Unodes[Unode_Index][:T_hat_U_array][room_num + 1, 1]
                    @linkconstraint(graph, (Unodes[Unode_Index + 1][:T_hat_U_array][room_num]) == 
                        dt_U/Ci*(Unodes[Unode_Index][:uT_U_array][room_num] + ((Tj1 − T_room_num)/Rij + (Tj2 − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                        (Unodes[Unode_Index][:T_hat_U_array][room_num])) # relationship between room temps
                end
            end
        end
        @objective(Unodes[Unode_Index], Min, 
            (0.01*(sum((Unodes[Unode_Index][:T_hat_U_array][i, 1] − Td_array[i]).^2 for i in (1:n_rooms))) 
            + (100*sum(Unodes[Unode_Index][:u_hat_U_array]).^2)).*dt_U)
        # 0.01∥T^u(s;tuk)−Td∥2+100∥u^(s;tk)∥2
    end



    # Link Constraints between Unodes and Lnodes
    for i in range(1,N_U) # loop through each U node
        for j in range(1,Int(N_L/N_U)) # loops through index of each subordinate Lnode at a time
            Lnode_Index = Int(j + N_L*(i - 1)/N_U)

            #create link constraint between them
            for room_index in range(1, n_rooms) # add a link constraint for each room

                # represent the room of the specific Lnode that the Unode i will link to 
                T_hat_l_room = Lnodes[Lnode_Index][:T_hat_L_array][room_index]

                # represent the upper layer temp solution for the lower layer
                T_0 = Unodes[i][:T_hat_U_array][room_index]
                
                # Makes T0 values the same between Unodes and Lnodes
                @linkconstraint(graph, T_0 == Lnodes[Lnode_Index][:T_0_LVal_array][room_index])
                
                @linkconstraint(graph, (T_hat_l_room) == 1/Ci*(ui*cp*(Ts − T_hat_l_room) 
                    + ((T_0 − T_hat_l_room)/Rij + (Toa − T_hat_l_room)/Rioa) + Pid))
            end
        end
    end


    # Link Constraints Between Lnodes
    for Lnode_Index in 1:(length(Lnodes)) # iterate through all Lnodes except for last
        
        for (room_num, T_room_num) in enumerate(Lnodes[Lnode_Index][:T_hat_L_array]) # iterate through each room
            if Lnode_Index <= length(Lnodes) - 1
            
                if room_num == 1 # if in the first room
                    Tj = Lnodes[Lnode_Index][:T_hat_L_array][room_num + 1] # establish neighboring room
                    @linkconstraint(graph, Lnodes[Lnode_Index + 1][:T_hat_L_array][room_num] == 
                        dt_L/Ci*(Lnodes[Lnode_Index][:uT_L_array][room_num] + ((Tj − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                        (Lnodes[Lnode_Index][:T_hat_L_array][room_num])) # relationship between room temps

                elseif room_num == n_rooms # if in the last last room
                    Tj = Lnodes[Lnode_Index][:T_hat_L_array][room_num - 1]
                    @linkconstraint(graph, (Lnodes[Lnode_Index + 1][:T_hat_L_array][room_num]) == 
                        dt_L/Ci*(Lnodes[Lnode_Index][:uT_L_array][room_num] + ((Tj − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                        (Lnodes[Lnode_Index][:T_hat_L_array][room_num])) # relationship between room temps

                else # if in a room surrounded by 2 rooms
                    Tj1 = Lnodes[Lnode_Index][:T_hat_L_array][room_num - 1]
                    Tj2 = Lnodes[Lnode_Index][:T_hat_L_array][room_num + 1]
                    @linkconstraint(graph, (Lnodes[Lnode_Index + 1][:T_hat_L_array][room_num]) == 
                        dt_L/Ci*(Lnodes[Lnode_Index][:uT_L_array][room_num] + ((Tj1 − T_room_num)/Rij + (Tj2 − T_room_num)/Rij + (Toa − T_room_num)/Rioa) + Pid) + 
                        (Lnodes[Lnode_Index][:T_hat_L_array][room_num])) # relationship between room temps
                end
            end
        end
        
        @objective(Lnodes[Lnode_Index], Min, 
            (c1.*sum(Lnodes[Lnode_Index][:T_hat_L_array] − Td_array).^2 + sum(Lnodes[Lnode_Index][:u_hat_L_array])^2)
             + c2.*sum(((Lnodes[Lnode_Index][:T_hat_L_array] − 
                    Lnodes[Lnode_Index][:T_0_LVal_array][:]).^2).*dt_L ))
        
    end

    # Set Optimizer
    set_optimizer(graph, Ipopt.Optimizer);
    set_optimizer_attribute(graph, "max_iter", 100);
    # Call the optimizer
    optimize!(graph);
    
    return graph, Unodes, Lnodes
    
end


HMPC_HVAC (generic function with 1 method)

In [31]:
graph, Unodes, Lnodes = HMPC_HVAC(2, 6, 5, 1, 2)
#   HMPC_HVAC(n_u, n_l, dtu, dtl, num_rooms)
println(Lnodes)

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:      144
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      118

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       24
                     variables with only upper bounds:        0
Total number of equality constraints.................:       52
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.0512000e+03 1.11e+04 1.38e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

OptiNode[OptiNode w/ 8 Variable(s) and 2 Constraint(s), OptiNode w/ 8 Variable(s) and 2 Constraint(s), OptiNode w/ 8 Variable(s) and 2 Constraint(s), OptiNode w/ 8 Variable(s) and 2 Constraint(s), OptiNode w/ 8 Variable(s) and 2 Constraint(s), OptiNode w/ 8 Variable(s) and 2 Constraint(s)]


In [8]:
# get temp graph for a room 
room_num = 1
temp_array = zeros(length(Lnodes))
for (i, Lnode) in enumerate(Lnodes)
    temp_array[i] = value(Lnode[:T_hat_l_array][room_num])
end

LoadError: UndefVarError: `Lnodes` not defined

In [9]:
using PlasmoPlots

plt_graph = layout_plot(graph,
                        node_labels=true,
                        markersize=20,
                        labelsize=4,
                        linewidth=2,
                        layout_options=Dict(:tol=>.01,
                                            :iterations=>100),
                        plt_options=Dict(:legend=>false,
                                         :framestyle=>:box,
                                         :grid=>false,
                                         :size=>(800,800),
                                         :axis => nothing))

LoadError: UndefVarError: `graph` not defined

In [10]:

N = 8

Tr1Arr = zeros(length(Lnodes))
Tr2Arr = zeros(length(Lnodes))


for (i, Lnode)  in enumerate(Lnodes)
    Tr1Arr[i] = value(Lnode[:T_hat_L_array][1])
    Tr2Arr[i] = value(Lnode[:u_hat_L_array][2])
end

xarray = Array{Array}(undef, 2)
xarray[2] = 0:10/(N-1):10

yarray = Array{Array}(undef, 2)
yarray[2] = 0:10/(N-1):10

zarray = Array{Array}(undef, 2)
zarray[2] = 0:10/(N-1):10


# Try this with more itterations

LoadError: UndefVarError: `Lnodes` not defined

In [11]:
plot((1:length(Tr1Arr)), Tr1Arr, title = "Temp of room 1 over time", xlabel = "Node (N)", ylabel = "Temperature (K)", label = ["Temperature of Room 1" "X Setpoint"])

LoadError: UndefVarError: `Tr1Arr` not defined

# LNode_Index Breakdown

In [12]:

N_U = 5 # number of Unodes
N_L = 25 # Number of Lnodes

for i in range(1,N_U) # loop through each U 
    println()
    print(i, " Unode")
    println()
        for j in range(1, Int(N_L/N_U)) # loop through 4 Lnodes at a time
            Lnode_Index = floor(Int, j + N_L*(i - 1)/N_U)
            print(Lnode_Index, " ")
    end
end


1 Unode
1 2 3 4 5 
2 Unode
6 7 8 9 10 
3 Unode
11 12 13 14 15 
4 Unode
16 17 18 19 20 
5 Unode
21 22 23 24 25 